# RAGEx for RAG

basierend auf: https://dl.acm.org/doi/pdf/10.1145/3626772.3657660

In [1]:
import sys, json
from pathlib import Path

# Füge das Projektverzeichnis (mit `src/`) dem Python-Pfad hinzu, egal von wo das Notebook gestartet wird.
project_root = next((p for p in [Path.cwd()] + list(Path.cwd().parents) if (p / 'src').exists()), None)
if project_root is None:
    raise RuntimeError("\"src\"-Verzeichnis nicht gefunden. Bitte Notebook im Projekt laufen lassen.")
root_str = str(project_root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)


In [3]:
from src.modules.rag_ex_explainable import RAGExExplainable
from src.modules.rag_engine import RAGEngine
from src.modules.multihop_rag_engine import MultiHopRAGEngine
from src.modules.llm_client import LLMClient
from src.modules.medmcqa_data_loader import MedMCQADataLoader
from src.modules.statspearls_data_loader import StatPearlsDataLoader

import tomllib

### Real data example

In [7]:
config_path = project_root / "config.toml"
config = {}

if config_path.exists():
    with open(config_path, "rb") as f:
        config = tomllib.load(f)

medmcqa_config = config.get("medmcqa") or {}
rag_config = config.get("rag") or {}
llm_config = config.get("llm") or {}

llm_model = llm_config.get("model", "gemma3:4b")
llm_provider = llm_config.get("provider", "ollama")

client = LLMClient(provider=llm_provider, model_name=llm_model)

SPLIT = medmcqa_config.get("split", "val")
PERSIST_DIR = project_root / "data" / "vector_db_statpearls"
NUM_HOPS = rag_config.get('n_hops', 2)

In [5]:
stat_loader = StatPearlsDataLoader(root_dir=str(project_root / "data"))
documents, stats = stat_loader.setup()

rag_engine = RAGEngine(persist_dir=str(PERSIST_DIR))
rag_engine.setup(documents=documents)

multi_hop = MultiHopRAGEngine(rag_engine=rag_engine, llm_client=client, num_hops=NUM_HOPS)

Creating new vector store with 13883 documents...
RagEngine ready.
Connecting to local Ollama (gemma3:4b)...


In [8]:
def format_medmcqa_question(item):
    question = str(item.get("question", "")).strip()
    options = []
    for label, key in [("A", "opa"), ("B", "opb"), ("C", "opc"), ("D", "opd")]:
        opt = str(item.get(key, "")).strip()
        if opt:
            options.append(f"{label}: {opt}")
    if options:
        question = f"{question}\n\nOptions:\n" + "\n".join(options)
    return question.strip()

med_loader = MedMCQADataLoader()
questions = med_loader.setup(split=SPLIT, as_documents=False, limit=1)

if not questions:
    raise RuntimeError("No MedMCQA questions loaded.")

results = []
for item in questions:
    question_text = format_medmcqa_question(item)
    if not question_text:
        continue

    trace, _ = multi_hop.run_and_trace(question_text)
    final_answer = (trace.get("final_answer") or "").strip()

    context_blocks = []
    for hop in trace.get("hops", []):
        doc = hop.get("retrieved_document_for_this_hop")
        if doc is None:
            continue
        content = getattr(doc, "page_content", None)
        if content is None:
            content = str(doc)
        context_blocks.append(str(content).strip())

    context = "\n\n".join([c for c in context_blocks if c])
    explainer = RAGExExplainable(llm_client=client)
    explanation = explainer.explain(query=question_text, answer=final_answer, context=context)

    results.append(
        {
            "question": question_text,
            "final_answer": final_answer,
            "trace": trace,
            "explanation": explanation,
        }
    )

print(results[0]["question"])
print(results[0]["final_answer"])
result = results[0]["explanation"]

--- Starting Multi-Hop Search for: 'Tensor veli palatini is supplied by:

Options:
A: Facial nerve
B: Trigeminal nerve
C: Glossopharyngeal nerve
D: Pharyngeal plexus' ---

[ Hop 1 ]
Executing search with query: 'Tensor veli palatini is supplied by:

Options:
A: Facial nerve
B: Trigeminal nerve
C: Glossopharyngeal nerve
D: Pharyngeal plexus'
Generating next query...

[ Hop 2 ]
Executing search with query: 'facial nerve anatomy'

Generating final answer...

--- Multi-Hop Search Complete. Final Answer: B: Trigeminal nerve ---
--- Multi-Hop Context: 

 ('<doc id="chunk-1-1" from_hop="1" search_query="Tensor veli palatini is supplied by:\n\nOptions:\nA: Facial nerve\nB: Trigeminal nerve\nC: Glossopharyngeal nerve\nD: Pharyngeal plexus">\nwith some fibers traveling with the internal maxillary artery to innervate the sweat glands of the face Gray rami communicantes: Some postganglionic fibers distribute to spinal nerves of vertebral levels C1 to C4 via gray rami communicantes, which essential

In [9]:
print(RAGExExplainable.prettify(result))

token	importance	strategies
with	0.787	leave_one_out:raw=0.589,sim=0.411 | random_noise:raw=0.588,sim=0.412
some	0.787	leave_one_out:raw=0.589,sim=0.411 | random_noise:raw=0.588,sim=0.412
fibers	0.781	leave_one_out:raw=0.585,sim=0.415 | random_noise:raw=0.582,sim=0.418
traveling	0.810	leave_one_out:raw=0.606,sim=0.394 | random_noise:raw=0.588,sim=0.412
with	0.796	leave_one_out:raw=0.596,sim=0.404 | random_noise:raw=0.583,sim=0.417
the	0.786	leave_one_out:raw=0.588,sim=0.412 | random_noise:raw=0.585,sim=0.415
internal	0.799	leave_one_out:raw=0.590,sim=0.410 | random_noise:raw=0.598,sim=0.402
maxillary	0.819	leave_one_out:raw=0.585,sim=0.415 | random_noise:raw=0.613,sim=0.387
artery	0.786	leave_one_out:raw=0.587,sim=0.413 | random_noise:raw=0.588,sim=0.412
to	0.781	leave_one_out:raw=0.584,sim=0.416 | random_noise:raw=0.584,sim=0.416
innervate	0.817	leave_one_out:raw=0.568,sim=0.432 | random_noise:raw=0.611,sim=0.389
the	0.786	leave_one_out:raw=0.588,sim=0.412 | random_noise:raw=0.585,sim